In [16]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

数据加载

In [17]:
market = pd.read_csv('Market_Basket_Optimisation.csv',header=None)
market.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


转换为可接受数据格式

In [18]:
market_hot_encode=pd.DataFrame(columns=["items"])

In [ ]:
# 在market_hot_encode逐行写入 xxx|xxx|xxx格式数据
for i in market.index:
    item=""
    for j in market.loc[i].unique():
        item+=str(j)+"|"
    market_hot_encode.loc[i,"items"]=item

In [14]:
market_hot_encode.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
market_hot_encode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7501 entries, 0 to 7500
Columns: 120 entries,  asparagus to zucchini
dtypes: int64(120)
memory usage: 7.2 MB


In [7]:
# one-hot编码,并删除nan列
market_hot_encode=market_hot_encode["items"].str.get_dummies(sep="|").drop("nan",axis=1)

In [8]:
market_hot_encode.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


频繁项集

In [9]:
# 挖掘频繁项集，最小支持度为0.025
itemsets = apriori(market_hot_encode,use_colnames=True, min_support=0.025)

In [10]:
# 按照支持度从大到小进行时候粗
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
itemsets

-------------------- 频繁项集 --------------------


,support,itemsets
30,0.238368,(mineral water)
12,0.179709,(eggs)
38,0.174110,(spaghetti)
16,0.170911,(french fries)
8,0.163845,(chocolate)
...,...,...
33,0.026530,(pepper)
5,0.025730,(cereals)
42,0.025730,(vegetables mix)
54,0.025463,"(green tea, eggs)"


关联规则

In [11]:
# 根据频繁项集计算关联规则，设置最小提升度为1.5
rules =  association_rules(itemsets, metric='lift', min_threshold=1.5)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(spaghetti),(ground beef),0.174110,0.098254,0.039195,0.225115,2.291162,0.022088,1.163716
5,(ground beef),(spaghetti),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997
12,(burgers),(eggs),0.087188,0.179709,0.028796,0.330275,1.837830,0.013128,1.224818
13,(eggs),(burgers),0.179709,0.087188,0.028796,0.160237,1.837830,0.013128,1.086988
16,(mineral water),(olive oil),0.238368,0.065858,0.027596,0.115772,1.757904,0.011898,1.056449
17,(olive oil),(mineral water),0.065858,0.238368,0.027596,0.419028,1.757904,0.011898,1.310962
2,(mineral water),(ground beef),0.238368,0.098254,0.040928,0.171700,1.747522,0.017507,1.088672
3,(ground beef),(mineral water),0.098254,0.238368,0.040928,0.416554,1.747522,0.017507,1.305401
15,(frozen vegetables),(spaghetti),0.095321,0.174110,0.027863,0.292308,1.678867,0.011267,1.167018
14,(spaghetti),(frozen vegetables),0.174110,0.095321,0.027863,0.160031,1.678867,0.011267,1.077039
